In [1]:
import numpy as np
import pylab as pl
import cosmocnc as cnc
import time

2025-03-05 09:48:06.649085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741164486.662606 1871911 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741164486.666414 1871911 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


defaulting to  /rds-d4/user/iz221/hpc-work/cosmopower/
defaulting to  /home/anicola_hpc/projects/ACT_clustering/code/cosmocnc/cosmocnc/../


In [2]:
zc_edges = np.array([0.2, 0.4, 0.55, 0.7, 0.85])

In [3]:
number_counts = cnc.cluster_number_counts()

number_counts.cnc_params["survey_sr"] = "/home/anicola_hpc/projects/ACT_clustering/scripts/survey_sr_act.py"
number_counts.cnc_params["survey_cat"] = None

number_counts.cnc_params["load_catalogue"] = False

number_counts.cnc_params["cluster_catalogue"] = "act"
number_counts.cnc_params["observables"] =  [["q_act"]]
number_counts.cnc_params["obs_select"] = "q_act"

number_counts.cnc_params["likelihood_type"] = "unbinned"
number_counts.cnc_params["data_lik_from_abundance"] = True
number_counts.cnc_params["compute_abundance_matrix"] = True

number_counts.cnc_params["obs_select_min"] = 5.
number_counts.cnc_params["obs_select_max"] = 200.
number_counts.cnc_params["M_min"] = 5e13
number_counts.cnc_params["M_max"] = 1e16

number_counts.cnc_params["n_points"] = 1024*16
number_counts.cnc_params["n_z"] = 100

number_counts.cnc_params["scalrel_type_deriv"] = "numerical"
number_counts.cnc_params["abundance_integral_type"] = "fft"
number_counts.cnc_params["apply_obs_cutoff"] = False

number_counts.cnc_params["cosmology_tool"] = "classy_sz"
number_counts.cnc_params["hmf_calc"] = "classy_sz"
number_counts.cnc_params["class_sz_hmf"] = "T08M200c"
number_counts.cnc_params["mass_definition"] = "200c"
number_counts.cnc_params["hmf_type_deriv"] = "numerical"
# number_counts.cnc_params["interp_tinker"] = "linear"


number_counts.cnc_params["number_cores_hmf"] = 1
number_counts.cnc_params["number_cores_abundance"] = 1#8
number_counts.cnc_params["number_cores_data"] = 1# 8
number_counts.cnc_params["parallelise_type"] = "redshift"

number_counts.cnc_params["cosmo_amplitude_parameter"] = "sigma_8"
number_counts.cnc_params["Hubble_parameter"] = "H0"
number_counts.cnc_params["cosmo_param_density"] = "physical"
number_counts.cnc_params["class_sz_cosmo_model"] = "mnu"
number_counts.cnc_params["cosmo_model"] = number_counts.cnc_params["class_sz_cosmo_model"]

#    "class_sz_cosmo_model": "lcdm", # lcdm, mnu, neff, wcdm, ede

number_counts.cnc_params["cosmocnc_verbose"] = "minimal"

number_counts.cnc_params["likelihood_type"] = "binned"
number_counts.cnc_params["obs_select_min"] = 5.000

number_counts.scal_rel_params["q_cutoff"] = 0.

number_counts.cnc_params["bins_edges_obs_select"] = np.exp(np.linspace(np.log(number_counts.cnc_params["obs_select_min"]), \
                                                                    np.log(number_counts.cnc_params["obs_select_max"]),7))

number_counts.scal_rel_params.update({
'A0' : np.log10(2.2e-05),
'B0': 0.08,
'SZmPivot' : 4.25e14, #in Msun
'bias_sz' : 1.,
'sigma_lnq_act' : 0.2,
'C0' : 2.,
"dof":0.,
}
)

number_counts.cosmo_params.update({

"Ob0h2":0.049*0.68**2,
"Oc0h2":(0.261)*0.68**2,
'H0': 68.,
'h': 0.68,
'sigma8': 0.81,
'tau_reio':  0.0561,
'n_s': 0.965,
"m_nu":0.000006,
"A_s":2.080447023199556e-09,
}
)

number_counts.initialise()

for i in range(len(zc_edges) - 1):

    number_counts.cnc_params.update({
            "z_min": zc_edges[i],
            "z_max": zc_edges[i+1],
            "bins_edges_z": np.linspace(zc_edges[i], \
                            zc_edges[i+1],9)
    })

    number_counts.update_params(number_counts.cosmo_params,number_counts.scal_rel_params, number_counts.cnc_params)

    number_counts.get_number_counts() #Evaluate the binned likelihood
    number_counts.get_bias_weighted_cluster_abundance() #Evaluate the binned likelihood

    n_tot = number_counts.n_tot
    n_tot_bias_weighted = number_counts.n_tot_bias_weighted

    bias_mean = n_tot_bias_weighted/n_tot

    print('Redshift bin edges: ', zc_edges[i], zc_edges[i+1])
    print('Average bias: ', bias_mean)

INFO - Cosmology params: {'Om0': 0.315, 'Ob0': 0.04897, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.080447023199556e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.00141808, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}


PATH_TO_CLASS_SZ_DATA not set. Setting it now...
Defaulting to: /home/anicola_hpc/class_sz_data_directory


2025-03-05 09:48:08.791156: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast
/home/anicola_hpc/.conda/envs/act-clust/lib/python3.11/site-packages/mcfit/mcfit.py:130: UserWarning: use backend='jax' if desired
  warnings.warn("use backend='jax' if desired")
INFO - computing class_szfast done
INFO - Cosmology params: {'Om0': 0.31, 'Ob0': 0.04900000000000001, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.0866006382888505e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.0013931690968152894, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast
INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 708.27464
INFO - Co

Redshift bin edges:  0.2 0.4
Average bias:  5.685251694733975


INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 623.63282
INFO - Collecting hmf
INFO - Cosmology params: {'Om0': 0.31, 'Ob0': 0.04900000000000001, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.0866006382888505e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.0013931690968152894, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast


Redshift bin edges:  0.4 0.55
Average bias:  6.498779864761472


INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 533.07343
INFO - Collecting hmf
INFO - Cosmology params: {'Om0': 0.31, 'Ob0': 0.04900000000000001, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.0866006382888505e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.0013931690968152894, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast


Redshift bin edges:  0.55 0.7
Average bias:  7.258837945208517


INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 396.78785
INFO - Collecting hmf


Redshift bin edges:  0.7 0.85
Average bias:  8.037982322087682
